In [0]:
%sql
use j2d;

In [0]:
spark.conf.set("spark.sql.adaptive.enabled", "false")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [0]:
from pyspark.sql import functions as F

In [0]:
df = (
    spark.range(start=1, end=1_00_000+1, numPartitions=5)
    .select(F.col("id"), F.rand(10).alias("value"))
)

df.display()

In [0]:
df.count()

In [0]:
df.rdd.getNumPartitions()

In [0]:
def count_per_partition(df):
    return df.rdd.mapPartitionsWithIndex(lambda idx, it: [(idx, sum(1 for i in it))]).collect()

partition_counts = count_per_partition(df)
for partition_id, row_count in partition_counts:
    print(partition_id, '-', row_count)

In [0]:
%sql
drop table if exists non_bucketed_table1;
drop table if exists non_bucketed_table2;

In [0]:
df.write.format("parquet").mode("overwrite").saveAsTable("non_bucketed_table1")
df.write.format("parquet").mode("overwrite").saveAsTable("non_bucketed_table2")

In [0]:
non_buct_df1 = spark.table("non_bucketed_table1")
non_buct_df2 = spark.table("non_bucketed_table2")

In [0]:
%fs
ls dbfs:/user/hive/warehouse/j2d.db/non_bucketed_table1

In [0]:
non_buct_join = (
    non_buct_df1
    .join(
        non_buct_df2,
        on=["id"],
        how="left"
    )
)
non_buct_join.show()

In [0]:
%sql
drop table if exists bucketed_table1;
drop table if exists bucketed_table2;

In [0]:
df.write.format("parquet").mode("overwrite").bucketBy(5, "id").sortBy("id").saveAsTable("bucketed_table1")
df.write.format("parquet").mode("overwrite").bucketBy(5, "id").sortBy("id").saveAsTable("bucketed_table2")

In [0]:
%fs
ls dbfs:/user/hive/warehouse/j2d.db/bucketed_table1

In [0]:
bucketed_df1 = spark.table("bucketed_table1")
bucketed_df2 = spark.table("bucketed_table2")

In [0]:
buct_join = (
    bucketed_df1
    .join(
        bucketed_df2,
        on=["id"],
        how="left"
    )
)
buct_join.show()